<a href="https://colab.research.google.com/github/thibaud-perrin/hibo-mistral-7b-fc/blob/main/mistral_7b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 hibo-mistral-7b-fc Training

This notebook focuses on fine-tuning the model `mistralai/Mistral-7B-v0.1` into an instruct and function calling model named: `thibaud-perrin/hibo-mistral-7b-fc-vx`.

## 📦 Installation of Required Packages

First, we need to install the necessary packages to work with Hugging Face's transformers and datasets, as well as any other libraries required for our project. We'll use the `!pip install` command to install these packages. This ensures that our environment has all the tools we need for training and interacting with the model.

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 📚 Import of All Required Packages

After installing the required packages, we'll import them into our Jupyter notebook. This includes importing libraries such as `transformers`, `datasets`, and any other utilities we need for processing data, training the model, and evaluating its performance.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login, Repository, login
from google.colab import userdata
from google.colab import drive

## 🔄 Google Drive Mount

To access and store large datasets or model files, we will mount Google Drive to our Colab environment. This allows us to seamlessly use files stored in Google Drive as if they were local files, making data management much easier.

In [3]:
drive.mount("/content/gdrive")
!ln -s "/content/gdrive/MyDrive/Colab Notebooks/Projects/mistral-7b-instruct" "/content/MistralNotebook"
!ls "./MistralNotebook"

Mounted at /content/gdrive
ln: failed to create symbolic link '/content/MistralNotebook/mistral-7b-instruct': File exists
mistral-7b-instruct	   results			     thibaud-perrin
mistral-7b-instruct.ipynb  test-hibo-mistral-7b-fc-vx.ipynb


## 🌐 Global Variables

In this section, we'll define global variables for our project. These include the base model name (`mistralai/Mistral-7B-v0.1`), the new model name (`thibaud-perrin/hibo-mistral-7b-fc-vx`), the dataset name (`thibaud-perrin/hibo-function-calling-v1`), and any necessary authentication tokens or secrets required for accessing APIs or services.

In [4]:
username = userdata.get('username')
email = userdata.get('email')
wandb_key = userdata.get('wandb_key')
hugginface_token = userdata.get('hugginface_token')
TRAINING = True
SAVING = True
N_DATA=5_000

REPO_NAME = 'thibaud-perrin'
MODEL_NAME = 'hibo-mistral-7b-fc-v1.2'

In [5]:
#Use a sharded model to fine-tune in the free version of Google Colab.
base_model = "mistralai/Mistral-7B-v0.1" #bn22/Mistral-7B-Instruct-v0.1-sharded


dataset_name, new_model = f"{REPO_NAME}/hibo-function-calling-v1", f"{REPO_NAME}/{MODEL_NAME}"

## 🔑 Login to Hugging Face

We need to authenticate with the Hugging Face Hub to access models and datasets. This involves logging in using our Hugging Face credentials, which will allow us to push models to the Hub at the end of this notebook.

In [6]:
login(token=hugginface_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
!git config --global user.email "$email"
!git config --global user.name "$username"

In [8]:
# !git config user.name

In [9]:
# !git config user.email

## 📂 Loading the Dataset

Here, we'll load the dataset `thibaud-perrin/hibo-function-calling-v1` from the Hugging Face Hub and select the train split.

In [10]:
# Loading a Gath_baize dataset
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# Example of a row
dataset["chat"][1]

[{'content': 'I was wondering about the topic of war crimes. Specifically, why was Winston Churchill never tried for war crimes, just because the allies won World War 2?',
  'role': 'user'},
 {'content': 'Winston Churchill was not tried for war crimes mainly for two reasons. First, at the time, the concept of war crimes was not as well-established as it is now. Second, Churchill was seen as a hero in his own country and the Allied powers, which made it politically difficult to try him.',
  'role': 'assistant'},
 {'content': "But wasn't he responsible for making some questionable decisions during the war?",
  'role': 'user'},
 {'content': 'Yes, some of his decisions during the war were controversial, but Churchill argued that they were necessary to win the war. For example, he ordered the bombing of cities like Dresden and Hamburg, which resulted in significant civilian casualties.',
  'role': 'assistant'},
 {'content': 'That sounds like it could be considered a war crime.',
  'role': '

## 🤖 Loading the Base Model (Mistral 7B)

We'll load the pre-trained `mistralai/Mistral-7B-v0.1` model from Hugging Face. This model will serve as the starting point for our fine-tuning process, where we'll adapt it to our specific task of instruction and function calling.

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 🔤 Loading the Tokenizer

The tokenizer responsible for converting text into a format understandable by the model will be loaded. This tokenizer must be compatible with the base model we're using.

In [13]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token #this is my mistake, because of the tokenizer set the eos_token attention mask as “0”. That is why my fine-tuned model, never stop
tokenizer.pad_token = tokenizer.unk_token # instead, should replace by this
tokenizer.padding_side = "right"
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

### ✍️ Adding a Custom Template to the Tokenizer

To improve the model's understanding of our specific task, we'll add a custom template to the tokenizer. This involves defining a pattern or structure that the inputs to the model will follow, making it easier for the model to interpret the data.

In [14]:
import textwrap

template = """\
{% if messages[0]['role'] == 'system' %}\
    {% set loop_messages = messages[1:] %}\
    {% set system_message = messages[0]['content'] %}\
{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}\
    {% set loop_messages = messages %}\
    {% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}\
{% else %}\
    {% set loop_messages = messages %}\
    {% set system_message = false %}\
{% endif %}\
{% for message in loop_messages %}\
    {% if loop.index0 == 0 and system_message != false %}\
        {% set content = '<<SYS>>\n' + system_message + '\n<</SYS>>\n\n' + message['content'] %}\
    {% else %}\
        {% set content = message['content'] %}\
    {% endif %}\
    {% if message['role'] == 'user' %}\
        {{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}\
    {% elif message['role'] == 'system' %}\
        {{ '<<SYS>>\n' + content.strip() + '\n<</SYS>>\n\n' }}\
    {% elif message['role'] == 'assistant' %}\
        {% if loop.last or loop_messages[loop.index0 + 1]['role'] != 'assistant' %}
            {{ '[ASST] '  + content.strip() + ' [/ASST]' + eos_token }}\
        {% else %}
            {{ '[ASST] '  + content.strip() + ' [/ASST]' }}\
        {% endif %}
    {% elif message['role'] == 'function_response' %}\
        {{ bos_token + '[FUNC] '  + content.strip() + ' [/FUNC]' }}\
    {% endif %}\
{% endfor %}"""
tokenizer.chat_template = textwrap.dedent(template).replace('    ', '').strip()

In [15]:
# display the template to verify that it has been saved
tokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 and system_message != false %}{% set content = '<<SYS>>\n' + system_message + '\n<</

### ➕ Adding New Column to the Dataset

Based on the custom template, we'll add a new column to our dataset. This column will contain the processed inputs that are ready to be fed into the model for training.

In [16]:
# Define a function to concatenate and apply the template
def concatenate_and_transform(row):
    # Concatenate 'system' and 'chat' lists
    combined_list = row['system'] + row['chat']
    # Apply the tokenizer.apply_chat_template function on the concatenated list
    transformed_data = tokenizer.apply_chat_template(combined_list, tokenize=False, add_generation_prompt=False)
    return {'chat_sample': transformed_data}

In [17]:
dataset = dataset.map(concatenate_and_transform)
# Select the first 200,000 rows after shuffling
dataset = dataset.select(range(N_DATA))

In [18]:
# example after adding the new column
dataset[35]

{'dataset_origin': 'glaiveai/glaive-function-calling-v2',
 'system': [{'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "get_stock_price",\n    "description": "Get the current stock price of a company",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "company_name": {\n                "type": "string",\n                "description": "The name of the company"\n            },\n            "exchange": {\n                "type": "string",\n                "description": "The stock exchange where the company is listed"\n            }\n        },\n        "required": [\n            "company_name",\n            "exchange"\n        ]\n    }\n}',
   'role': 'system'}],
 'chat': [{'content': 'Hi, can you tell me the current stock price of Apple on NASDAQ?',
   'role': 'user'},
  {'content': 'Sure, let me fetch that information for you.',
   'role': 'assistant'},
  {'content': '<functionca

## 💪 Train the Model

### 📊 Login to Weights & Biases

Weights & Biases (W&B) is a tool for tracking experiments, and we'll log in to it to monitor our model's training progress. This helps in visualizing metrics and comparing different training runs.

In [19]:
if TRAINING:
  wandb.login(key = wandb_key)
  run = wandb.init(project='Fine tuning mistral 7B for FC', job_type="training", anonymous="allow")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thibaud-perrin. Use `wandb login --relogin` to force relogin


### ⚙️ Init LoRA Config

LoRA (Low-Rank Adaptation) is a technique for efficient model fine-tuning. We'll initialize the LoRA configuration, specifying any parameters needed to adapt the model to our task with minimal computational resources.

In [20]:
if TRAINING:
  model = prepare_model_for_kbit_training(model)
  peft_config = LoraConfig(
          r=16,
          lora_alpha=16,
          lora_dropout=0.05,
          bias="none",
          task_type="CAUSAL_LM",
          target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
      )
  model = get_peft_model(model, peft_config)

### 🏋️ Init Training Arguments and SFTTrainer

We'll initialize the training arguments, specifying details like the number of epochs, batch size, and learning rate. Then, we'll set up the `SFTTrainer` (Sparse Fine-Tuning Trainer) with these arguments to start the training process.

In [21]:
if TRAINING:
  # Training Arguments
  # Hyperparameters should beadjusted based on the hardware you using
  training_arguments = TrainingArguments(
      output_dir= "./MistralNotebook/results",
      resume_from_checkpoint="./MistralNotebook/results",
      num_train_epochs= 1,
      per_device_train_batch_size= 22,
      gradient_accumulation_steps= 2,
      optim = "paged_adamw_8bit",
      save_steps= 125,
      evaluation_strategy="steps", # Evaluate the model every logging step
      eval_steps=125,              # Evaluate and save checkpoints every 50 steps
      do_eval=True,                # Perform evaluation at the end of training
      logging_steps= 30,
      learning_rate= 2e-4,
      weight_decay= 0.001,
      fp16= False,
      bf16= False,
      max_grad_norm= 0.3,
      max_steps= -1,
      warmup_ratio= 0.3,
      group_by_length= True,
      lr_scheduler_type= "constant",
      report_to="wandb"
  )
  # Setting sft parameters
  trainer = SFTTrainer(
      model=model,
      train_dataset=dataset,
      peft_config=peft_config,
      max_seq_length= None,
      dataset_text_field="chat_sample",
      tokenizer=tokenizer,
      args=training_arguments,
      packing= False,
  )

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### 🚀 Start Training, and Save the Model

In this section, we'll start the training process using the data and configurations we've set up. After training, we'll save the fine-tuned model so it can be used for inference or further training in the future.

In [22]:
if TRAINING:
  trainer.train(resume_from_checkpoint=False)
  # Save the fine-tuned model
  trainer.model.save_pretrained(f"./MistralNotebook/{new_model}")
  trainer.model.save_pretrained(f"./{REPO_NAME}-1/{MODEL_NAME}")
  wandb.finish()
  model.config.use_cache = True
  model.eval()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,0.787700


Step,Training Loss
30,0.787700
60,0.655200
90,0.574300


train/epoch,▁▄▆█
train/global_step,▁▃▆█
train/grad_norm,█▂▁
train/learning_rate,▁▁▁
train/loss,█▄▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0


## 🧪 Test the Model

We'll evaluate the performance of our fine-tuned model on a test set or through specific examples to ensure it performs well on the task of instruction and function calling.

In [23]:
def stream(user_prompt, model, tokenizer):
    runtimeFlag = "cuda:0"
    system_prompt = '<<SYS>>\nThe conversation between Human and AI assistant\n<</SYS>>'
    B_INST, E_INST = "[INST]", " [/INST]"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200)

In [24]:
stream("Explain large language models", model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[ASST] Large language models are a type of artificial intelligence that can generate human-like text. They are trained on massive amounts of data, such as books, articles, and social media posts, and can be used for a variety of tasks, including translation, summarization, and question answering. [/ASST]


In [25]:
import shutil

# Correct the directory name if it was a typo
source_dir = f'./MistralNotebook/{REPO_NAME}/{MODEL_NAME}'  # Ensure this is the correct path
destination_dir = MODEL_NAME  # Specify your destination directory

# Copy the directory
try:
    shutil.copytree(source_dir, f"./{destination_dir}")
    shutil.copytree(source_dir, f"./{REPO_NAME}-copy/{destination_dir}")
    print(f"Directory copied successfully from {source_dir} to {destination_dir}.")
except Exception as e:
    print(f"Error copying directory: {e}")

Directory copied successfully from ./MistralNotebook/thibaud-perrin/hibo-mistral-7b-fc-v1.2 to hibo-mistral-7b-fc-v1.2.


## ♻️ Clear the Memory Footprint and Reload the Model

To ensure our environment is clean and to avoid any memory issues, we'll clear any unnecessary data from memory and then reload our fine-tuned model for further use or evaluation.

In [26]:
# Clear the memory footprint
if TRAINING:
  del model, trainer
torch.cuda.empty_cache()

# Reload the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})

if TRAINING:
  model = PeftModel.from_pretrained(base_model_reload, new_model) # f"./{REPO_NAME}-copy/{destination_dir}"
else:
  model = PeftModel.from_pretrained(base_model_reload, f"./{REPO_NAME}-copy/{destination_dir}")
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 🔁 Reload the Tokenizer

After clearing the memory, we'll reload the tokenizer associated with our fine-tuned model to ensure it's ready for processing new inputs.

In [27]:
# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"
tokenizer.chat_template = textwrap.dedent(template).replace('    ', '').strip()
tokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 and system_message != false %}{% set content = '<<SYS>>\n' + system_message + '\n<</

In [35]:
stream("As a data scientist, can you explain the concept of regularization in machine learning", model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Regularization is a technique used in machine learning to prevent overfitting and improve the generalization performance of a model. It involves adding a penalty term to the loss function that penalizes complex models with many parameters. This helps to prevent the model from fitting the training data too closely and improves its ability to make accurate predictions on new data. [/ASST]


## 🚚 Push the Model and the Tokenizer to HuggingFace Repo

Finally, we'll push our fine-tuned model and its tokenizer to the Hugging Face Hub under the repository `thibaud-perrin/hibo-mistral-7b-fc-vx`. This allows others to access and use our model for their own tasks.

In [29]:
assert SAVING == True, "Do not save this model"

In [32]:
#push the model to hub
model.push_to_hub(f"{REPO_NAME}/{MODEL_NAME}", use_temp_dir=False)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/thibaud-perrin/hibo-mistral-7b-fc-v1.2/commit/cd8be6d8d757f201c9e2f41d278faa98dd0fdcf5', commit_message='Upload MistralForCausalLM', commit_description='', oid='cd8be6d8d757f201c9e2f41d278faa98dd0fdcf5', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
tokenizer.push_to_hub(f"{REPO_NAME}/{MODEL_NAME}", use_temp_dir=False)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/thibaud-perrin/hibo-mistral-7b-fc-v1.2/commit/eda45756aee0605a84f44c585ff5ed1b0d51a83e', commit_message='Upload tokenizer', commit_description='', oid='eda45756aee0605a84f44c585ff5ed1b0d51a83e', pr_url=None, pr_revision=None, pr_num=None)

## Other technique to push on HF

In [ ]:
assert "a" == "b", "stop ! do not execute this if you do not want"

In [ ]:
from huggingface_hub import HfFolder, Repository


In [ ]:
repo_path = "./tokenizer"  # Local path to clone the repository
repo_url = "thibaud-perrin/hibo-mistral-7b-fc-v2"  # Your repository name on Hugging Face Hub

In [ ]:
# Clone the repository locally (if not already cloned)
repo = Repository(repo_path, clone_from=repo_url)

In [ ]:
# Reload tokenizer
tokenizer.padding_side = "right"
tokenizer.chat_template = textwrap.dedent(template).replace('    ', '').strip()
tokenizer.chat_template
# Save your tokenizer locally in the cloned repository directory
tokenizer.save_pretrained(repo_path)

In [ ]:
# Push changes
repo.push_to_hub()